# HANC with a Welfare State

In [ ]:
%load_ext autoreload
%autoreload 2

import time
import pickle
import numpy as np
from scipy import optimize
from consav.misc import elapsed

import sys
import os
original_stdout = sys.stdout

import matplotlib.pyplot as plt   
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
plt.rcParams.update({"axes.grid" : True, "grid.color": "black", "grid.alpha":"0.25", "grid.linestyle": "--"})
plt.rcParams.update({'font.size': 14})

from HANCWelfareModel import HANCWelfareModelClass
from steady_state import obj_ss

## Model without a Governement

I import the model and draw the DAG

In [ ]:
model = HANCWelfareModelClass(name='baseline')
par = model.par
ss = model.ss

In [ ]:
model.info(only_blocks=True)
model.draw_DAG()

I set all the steady values for the household problem manually at ad hoc values and now solve and simulate:

In [ ]:
ss.r = 0.02*(1-0.1)
ss.wt = 1.00*(1-0.3)
ss.w = ss.wt/(1-par.tau_ss)
ss.tau = par.tau_ss
ss.chi = par.chi_ss
ss.Gamma_Y = par.Gamma_Y

In [ ]:
model.solve_hh_ss(do_print=True)
model.simulate_hh_ss(do_print=True)

And check whether the results make sort of sense:

In [ ]:
print(f'{model.ss.A_hh = :.2f}')
print(f'{model.ss.L_hh = :.2f}')
print(f'{model.ss.C_hh = :.2f}')

The results seem to make sense and thus I will now solve for the steady state in the model with no Gorvernment.

In [ ]:
#setting values to zero to remove government
model.par.tau_ss = 0.0
model.par.chi_ss = 0.0

#simulating the steady state
model.find_ss(do_print=True)

I see we find a steady state where we see there is no government (ss.G=ss.Lg=0). Testing the stationary equilibrium I find

In [ ]:
model.test_path()

In [ ]:
model.test_hh_path()

And we see that the steady state is indeed a stationary equilibrium.

I can therefore calculate the expected utility of the household in the steady state:

In [ ]:
util_no_govern = np.sum([par.beta**t * ((np.sum((ss.u+(ss.G+par.S)**(1-par.omega)/(1-par.omega)) * ss.D / (np.sum(ss.D))))) for t in range(par.T)])
print(f'{util_no_govern = :.2f}')

We see that the expected utility is very negavtive in the case of no Governement in the model. This is because the household is not insured against the risk of unemployment and thus the household is very exposed to the risk of unemployment. This is also why the household chooses to work a lot in the steady state.

## Optimal welfare policy with only taxes on labour income

I start by creating a new instance of our baseline model

In [ ]:
modelI = model.copy()

In [ ]:
optI = modelI.optimize_social_welfare(tau_guess=0.5,do_print=True)

To find the optimal policy with only a tax on labour income, $\tau$, I first define the objective function of the planner:

Next I use a scalar minimizer to find the optimal tax rate on labour income:

Thus I find the optimal tax on labour as $\tau=65.51\%$.

Adding this to the model and solving for the steady state I find:

The Goverment spending output ratio can therefore be calculated as:

In [ ]:
print(f'{modelI.ss.G/modelI.ss.Y = :.4f}')

## Optimal welfare policy with taxes on labour income and lumpsum transfers

I again create a new instance of the baseline model

In [ ]:
modelII = model.copy()

I run the optimize_social_welfare function now guessing on both $\tau$ and $\chi$ using the previous results as initial guesses:

In [ ]:
optII = modelII.optimize_social_welfare(tau_guess=optI[0],chi_guess=optI[1],do_print=True)

Thus the optimal taxrate in this case is $\tau = 47.66\%$ and the optimal lumpsum transfer is $\chi = -0.2181$ implying a lumpsum tax.

The Goverment spending output ratio can therefore be calculated as:

In [ ]:
ratio = modelII.ss.G/modelII.ss.Y
print(f'G_ss/Y_ss = {ratio :.4f}')

## Model with increased TFP and Government

I once again create a new instance of the model and repeat the same steps as before but with $\Gamma^Y = 1.1$:

In [ ]:
modelIII = model.copy()

modelIII.par.Gamma_Y = 1.1

Again I run the optimize_social_welfare function guessing on both $\tau$ and $\chi$ using the previous results as initial guesses:

In [ ]:
optIII = modelIII.optimize_social_welfare(tau_guess=optII[0],chi_guess=optII[1],do_print=True)

In [ ]:
ratio = modelIII.ss.G/modelIII.ss.Y
print(f'G_ss/Y_ss = {ratio :.4f}')

## Transition to new steady state

To find the transition to the new steady state I first compute the jabobian of the model:

In [ ]:
modelIII.compute_jacs(do_print=True)

Next I define the initial values for where the economy starts:

In [ ]:
ini = vars(modelII.ss)

To evaluate how the tranistion should be I will test out 4 different scenarios:
Common for all scenarios is that the governement observes the change in TFP in period 0 and will not adjust the tax rate and lumpsum transfer to the optimal values in period before period 1.


- In the baseline scenario the government jumps straight to the optimal tax rate and lumpsum transfer in period 1.
- In the second scenario the government will adjust the tax rate linearly from the initial tax rate to the optimal taxe rate in over an unspecified number of periods.
- In the third scenario the government will reduce the gap between the initial tax rate and the optimal tax rate by an unkonwn fraction in each period thereby the change in the tax rate will be decreasing over time.
- In the fourth scenario the government will increase the taxes from the initial tax rate to the optimal tax rate by an unknown fraction in each period thereby the change in the tax rate will be increasing over time.

I start by evaluating the baseline scenario:

In [ ]:
# Creating arrays for the shocks
dtau = np.zeros(par.T)
dchi = np.zeros(par.T)

# Setting shock values
dtau[0:1] = modelII.ss.tau - modelIII.ss.tau
dchi[0:1] = modelII.ss.chi - modelIII.ss.chi

# Creating dictionary for the shocks
shocks = {'dtau': dtau,
          'dchi': dchi}

modelIII.find_transition_path(shocks=shocks,ini=ini,do_print=False)
v_shock = np.sum([modelIII.par.beta**t * np.sum((modelIII.path.u[t]+(modelIII.path.G[t]+modelIII.par.S)**(1-modelIII.par.omega)/(1-modelIII.par.omega)) * modelIII.path.D[t] / (np.sum(modelIII.path.D[t]))) for t in range(modelIII.par.T)])
print(v_shock)

next I define an integer minimizer to find the optimal number of periods it takes for the economy to reach the new steady state in the second scenario and evaluate the results:

In [ ]:
def int_minimize(max_int, step_size=10,do_print=False,print_results=True):
    
    if do_print:
        print_results = True
    t0 = time.time()

    # Defining the objective function
    def obj_lin(len):
        #a. Creating arrays for the shocks
        dtau = np.zeros(par.T)
        dchi = np.zeros(par.T)

        #b. Assuming government cannot adjust the tax rate in the first period but must observe the shock
        dtau[0:1] = modelII.ss.tau - modelIII.ss.tau
        dchi[0:1] = modelII.ss.chi - modelIII.ss.chi

        #c. Setting the shock values for the remaining periods
        for i in range(1,len+1):
            dtau[i] = dtau[i-1] + (modelIII.ss.tau-modelII.ss.tau)/len
            dchi[i] = dchi[i-1] + (modelIII.ss.chi-modelII.ss.chi)/len

        #d. Creating dictionary for the shocks
        shocks = {'dtau': dtau,
                'dchi': dchi}        
        
        #e. finding the transition path
        sys.stdout = open(os.devnull, 'w')
        modelIII.find_transition_path(shocks=shocks,ini=ini,do_print=False)
        sys.stdout = original_stdout
        
        #f. Calculating the expected utility
        val = - np.sum([modelIII.par.beta**t * np.sum((modelIII.path.u[t]+(modelIII.path.G[t]+modelIII.par.S)**(1-modelIII.par.omega)/(1-modelIII.par.omega)) * modelIII.path.D[t] / (np.sum(modelIII.path.D[t]))) for t in range(modelIII.par.T)])
        
        return val
    
    # Rough search
    best_x = 0
    if do_print:
        print(f'Starting in x = {best_x}...', end='')
    best_val = obj_lin(best_x)
    if do_print:
        print(f'Value = {best_val:.5f}')
    for x in range(step_size, max_int + 1, step_size):
        if do_print:
            print(f'Checking x = {x}...', end='')
        val = obj_lin(x)
        if do_print:
            print(f'Value = {val:.5f}')

        if val < best_val:
            best_x = x
            best_val = val

    # Fine search
    start_x = max(0, best_x - step_size + 1)
    end_x = min(max_int, best_x + step_size - 1)
    if do_print:
        print(f'{best_x} Identified as best in rough search. Starting fine search from {start_x} to {end_x}...')
    for x in range(start_x, end_x + 1):
        if do_print:
            print(f'Checking x = {x}...', end='')
        val = obj_lin(x)
        if do_print:
            print(f'Value = {val:.5f}')
        if val < best_val:
            best_x = x
            best_val = val
    
    if print_results:
        print(f'Best x = {best_x} with expected utility of = {-best_val:.5f}')
        print(f'Found in: {elapsed(t0)}')
    return best_x, best_val

# def obj_nonlin_decrease(decr):
#     #a. Creating arrays for the shocks
#     dtau = np.zeros(par.T)
#     dchi = np.zeros(par.T)

#     #b. Assuming government cannot adjust the tax rate in the first period but must observe the shock
#     dtau[0:1] = modelII.ss.tau - modelIII.ss.tau
#     dchi[0:1] = modelII.ss.chi - modelIII.ss.chi
    
#     #c. Setting the shock values for the remaining periods
#     for i in range(1,par.T):
#         dtau[i] = dtau[i-1]*decr
#         dchi[i] = dchi[i-1]*decr
    
#     #d. Creating dictionary for the shocks
#     shocks = {'dtau': dtau,
#             'dchi': dchi}        
    
#     #e. finding the transition path
#     sys.stdout = open(os.devnull, 'w')
#     modelIII.find_transition_path(shocks=shocks,ini=ini,do_print=False)
#     sys.stdout = original_stdout

#     #f. Calculating the expected utility
#     val = - np.sum([modelIII.par.beta**t * np.sum((modelIII.path.u[t]+(modelIII.path.G[t]+modelIII.par.S)**(1-modelIII.par.omega)/(1-modelIII.par.omega)) * modelIII.path.D[t] / (np.sum(modelIII.path.D[t]))) for t in range(modelIII.par.T)])
    
#     return val

# def obj_nonlin_increase(incr):
#     #a. Creating arrays for the shocks
#     dtau = np.zeros(par.T)
#     dchi = np.zeros(par.T)

#     #b. Assuming government cannot adjust the tax rate in the first period but must observe the shock
#     dtau[0:1] = modelII.ss.tau - modelIII.ss.tau
#     dchi[0:1] = modelII.ss.chi - modelIII.ss.chi

#     #c. Setting the shock values for the remaining periods
#     for i in range(1,par.T):
#         if np.sign(dtau[i-1]) == np.sign((modelIII.ss.tau+dtau[i-1])*incr - modelIII.ss.tau):
#             dtau[i] = (modelIII.ss.tau+dtau[i-1])*incr - modelIII.ss.tau
        
#         if np.sign(dchi[i-1]) == np.sign((modelIII.ss.chi+dchi[i-1])*incr - modelIII.ss.chi):
#             dchi[i] = (modelIII.ss.chi+dchi[i-1])*incr - modelIII.ss.chi
    
#     #d. Creating dictionary for the shocks
#     shocks = {'dtau': dtau,
#             'dchi': dchi}        
    
#     #e. finding the transition path
#     sys.stdout = open(os.devnull, 'w')
#     modelIII.find_transition_path(shocks=shocks,ini=ini,do_print=False)
#     sys.stdout = original_stdout
    
#     #f. Calculating the expected utility
#     val = - np.sum([modelIII.par.beta**t * np.sum((modelIII.path.u[t]+(modelIII.path.G[t]+modelIII.par.S)**(1-modelIII.par.omega)/(1-modelIII.par.omega)) * modelIII.path.D[t] / (np.sum(modelIII.path.D[t]))) for t in range(modelIII.par.T)])
    
#     return val

In [ ]:
opt_lin = int_minimize(100, step_size=10,print_results=True)

In [ ]:
def obj_nonlin_decrease_(decr_tau,decr_chi):
    #a. Creating arrays for the shocks
    dtau = np.zeros(par.T)
    dchi = np.zeros(par.T)

    #b. Assuming government cannot adjust the tax rate in the first period but must observe the shock
    dtau[0:1] = modelII.ss.tau - modelIII.ss.tau
    dchi[0:1] = modelII.ss.chi - modelIII.ss.chi
    
    #c. Setting the shock values for the remaining periods
    for i in range(1,par.T):
        dtau[i] = dtau[i-1]*decr_tau
        dchi[i] = dchi[i-1]*decr_chi
    
    #d. Creating dictionary for the shocks
    shocks = {'dtau': dtau,
            'dchi': dchi}        
    
    #e. finding the transition path
    sys.stdout = open(os.devnull, 'w')
    modelIII.find_transition_path(shocks=shocks,ini=ini,do_print=False)
    sys.stdout = original_stdout

    #f. Calculating the expected utility
    val = - np.sum([modelIII.par.beta**t * np.sum((modelIII.path.u[t]+(modelIII.path.G[t]+modelIII.par.S)**(1-modelIII.par.omega)/(1-modelIII.par.omega)) * modelIII.path.D[t] / (np.sum(modelIII.path.D[t]))) for t in range(modelIII.par.T)])
    
    return val

In [ ]:
def obj_nonlin_increase_(incr_tau,incr_chi):
    #a. Creating arrays for the shocks
    dtau = np.zeros(par.T)
    dchi = np.zeros(par.T)

    #b. Assuming government cannot adjust the tax rate in the first period but must observe the shock
    dtau[0:1] = modelII.ss.tau - modelIII.ss.tau
    dchi[0:1] = modelII.ss.chi - modelIII.ss.chi

    #c. Setting the shock values for the remaining periods
    for i in range(1,par.T):
        if np.sign(dtau[i-1]) == np.sign((modelIII.ss.tau+dtau[i-1])*incr_tau - modelIII.ss.tau):
            dtau[i] = (modelIII.ss.tau+dtau[i-1])*incr_tau - modelIII.ss.tau
        
        if np.sign(dchi[i-1]) == np.sign((modelIII.ss.chi+dchi[i-1])*incr_chi - modelIII.ss.chi):
            dchi[i] = (modelIII.ss.chi+dchi[i-1])*incr_chi - modelIII.ss.chi
    
    #d. Creating dictionary for the shocks
    shocks = {'dtau': dtau,
            'dchi': dchi}        
    
    #e. finding the transition path
    sys.stdout = open(os.devnull, 'w')
    modelIII.find_transition_path(shocks=shocks,ini=ini,do_print=False)
    sys.stdout = original_stdout
    
    #f. Calculating the expected utility
    val = - np.sum([modelIII.par.beta**t * np.sum((modelIII.path.u[t]+(modelIII.path.G[t]+modelIII.par.S)**(1-modelIII.par.omega)/(1-modelIII.par.omega)) * modelIII.path.D[t] / (np.sum(modelIII.path.D[t]))) for t in range(modelIII.par.T)])
    
    return val

In [ ]:
opt_nonlin_decrease_ = optimize.minimize(lambda x: obj_nonlin_decrease_(x[0],x[1]), x0=[0.5,0.5], bounds=((0.0,1.0),(0.0,1.0)), method='Nelder-Mead')
print(f'tau = {opt_nonlin_decrease_.x[0]}')
print(f'chi = {opt_nonlin_decrease_.x[1]}')
print(f'Expected utility = {-opt_nonlin_decrease_.fun:.5f}')

In [ ]:
# opt_nonlin_decrease = optimize.minimize_scalar(obj_nonlin_decrease, bounds=(0.0,1.0), method='bounded')
# print(opt_nonlin_decrease)

In [ ]:
# opt_nonlin_increase = optimize.minimize_scalar(obj_nonlin_increase, bounds=(1.0,max(modelIII.ss.tau/modelII.ss.tau,modelIII.ss.chi/modelII.ss.chi)), method='bounded')
# print(opt_nonlin_increase)

In [ ]:
opt_nonlin_increase_ = optimize.minimize(lambda x: obj_nonlin_increase_(x[0],x[1]), x0=(1.0,1.0), bounds=((1.0,modelIII.ss.tau/modelII.ss.tau),(1.0,modelIII.ss.chi/modelII.ss.chi)),  method='L-BFGS-B')
print(f'tau increase = {opt_nonlin_increase_.x[0]}')
print(f'chi decrease = {opt_nonlin_increase_.x[1]}')
print(f'Expected utility = {-opt_nonlin_increase_.fun:.5f}')

In [ ]:
paths = ['Y','L','Lg','L_hh','K','G','C_hh','U_hh','rK','w','wt']
modelIII.show_IRFs(paths,ncols=4,T_max=100,abs_diff=['tau','chi'])

In [ ]:
def plot_model_results(models):
    fig, ax = plt.subplots(figsize=(12,4), dpi=100)

    # b. assets
    ax.set_title('Savings Comparison')

    for model in models:
        y = np.insert(np.cumsum(np.sum(model.ss.D, axis=(0,1))), 0, 0.0)
        label = f'tau={model.ss.tau:.3f}, chi={model.ss.chi:.3f}'
        ax.plot(np.insert(model.par.a_grid, 0, model.par.a_grid[0]), y/y[-1], label=label)

    ax.set_xlabel('assets, $a_{t}$')
    ax.set_ylabel('CDF')
    ax.set_xscale('symlog')
    ax.legend(loc='lower right')

# Call the function with all models
plot_model_results([model, modelI, modelII, modelIII])

In [ ]:
#vars(modelIII.ss).keys()